In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sachin-tendulkar-international-career-stats:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5298149%2F8822877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240727%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240727T164941Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9eeb96a47be0fd162d67e0f002d2c693e7e537fa0525eaf98f770c515c703e0b68b334d80368739740c7242497a04861246285f9071ece56827122df0aca0959f456b786efa1977912bb595d31f62287db1d17a6f946326089c2270513dcf1a9b3bc0126745a9be0f4ad73972f3f773059c8be00c405be5a28d41c3e235feba68d3e9751cf7d14c557d61c57809bf1b19baea3296cd36c616c13952856058deb1b11a99afac0bcfbdd44cba209a104373465007e0aac082efeeb2141218fb2b1e041a4396825d76b0b7ed7dca208ce8b841d62250d58f896f44a1603e0eeda8f6320f55b08add4427090005847328eff019588e66ff6d060e03fdbb777f23378'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')



[==================================================] 31267 bytes downloaded

Downloaded and uncompressed: sachin-tendulkar-international-career-stats

Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_odi_bowling.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_odi_fielding.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_t20_innings.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_t20_bowling.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_test_fielding.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_test_bowling.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_t20_fielding.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_test_innings.csv

/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_odi_innings.csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('/kaggle/input/sachin-tendulkar-international-career-stats/sachin_tendulkar_t20_innings.csv')
print(data.head())

   Runs  Mins  BF  4s  6s     SR  Pos Dismissal  Inns    Opposition  \

0    10    18  12   2   0  83.33    2    bowled     2  South Africa   



         Ground       Date  Unnamed: 12  

0  Johannesburg  12/1/2006          NaN  


In [5]:
#identify the features
features = data.columns
print(features)

Index(['Runs', 'Mins', 'BF', '4s', '6s', 'SR', 'Pos', 'Dismissal', 'Inns',

       'Opposition', 'Ground', 'Date', 'Unnamed: 12'],

      dtype='object')


In [6]:
#identify the data types
data_types = data.dtypes
print(data_types)

Runs             int64

Mins             int64

BF               int64

4s               int64

6s               int64

SR             float64

Pos              int64

Dismissal       object

Inns             int64

Opposition      object

Ground          object

Date            object

Unnamed: 12    float64

dtype: object


In [7]:
#summary
summary = data.describe()
print(summary)

       Runs  Mins    BF   4s   6s     SR  Pos  Inns  Unnamed: 12

count   1.0   1.0   1.0  1.0  1.0   1.00  1.0   1.0          0.0

mean   10.0  18.0  12.0  2.0  0.0  83.33  2.0   2.0          NaN

std     NaN   NaN   NaN  NaN  NaN    NaN  NaN   NaN          NaN

min    10.0  18.0  12.0  2.0  0.0  83.33  2.0   2.0          NaN

25%    10.0  18.0  12.0  2.0  0.0  83.33  2.0   2.0          NaN

50%    10.0  18.0  12.0  2.0  0.0  83.33  2.0   2.0          NaN

75%    10.0  18.0  12.0  2.0  0.0  83.33  2.0   2.0          NaN

max    10.0  18.0  12.0  2.0  0.0  83.33  2.0   2.0          NaN


In [8]:
print("SACHIN TENDULKAR'S T20 INNINGS RECORDS")

SACHIN TENDULKAR'S T20 INNINGS RECORDS
